In [1]:
import pandas as pd
import numpy as np
import openpyxl
import xlwings as xw
import datetime
import os

In [8]:
fname = r'F:\repos\fish\实时流.xlsx'
sheetname = '流水账'
df = pd.read_excel(fname,sheetname)
df.dtypes


日期      datetime64[ns]
单据号              int64
供货单位            object
品名              object
单位              object
入库数量             int64
入库单价           float64
入库金额           float64
期间              object
送货日期    datetime64[ns]
白云期间            object
dtype: object

In [9]:
df


,日期,单据号,供货单位,品名,单位,入库数量,入库单价,入库金额,期间,送货日期,白云期间
0,2014-03-24,90021129,驻马店白云纸业有限公司,55白云R级双胶787*1092,令,300,118.40,35520.90,2014-03,2014-03-23,2014-03
1,2014-03-24,90021129,驻马店白云纸业有限公司,55白云R级卷筒815,公斤,28665,5.11,146478.15,2014-03,2014-03-23,2014-03
2,2014-03-24,90021130,驻马店白云纸业有限公司,55白云双胶787*1092,令,180,124.08,22333.50,2014-03,2014-03-23,2014-03
3,2014-03-24,90021130,驻马店白云纸业有限公司,72白云双胶787*1092,令,352,167.06,58806.00,2014-03,2014-03-23,2014-03
4,2014-03-25,90021131,驻马店白云纸业有限公司,55白云R级双胶787*1092,令,1230,120.77,148542.59,2014-03,2014-03-24,2014-03
5,2014-03-25,90021131,驻马店白云纸业有限公司,55白云双胶787*1092,令,390,124.08,48389.25,2014-03,2014-03-24,2014-03


In [11]:
app = xw.App(visible=False, add_book=False)
wb = app.books.open(fname)
ws = wb.sheets[sheetname]
ws.clear()
wb.save()
wb.close()
app.quit()

with pd.ExcelWriter(fname,engine = 'openpyxl',date_format="YYYY-MM-DD",
    datetime_format="YYYY-MM-DD HH:MM:SS",mode='a',
                    if_sheet_exists='replace') as writer:    #overlay
    df.to_excel(writer, sheetname, index=False)
os.startfile(fname)

In [11]:
os.startfile(fname)

In [5]:
def qijianJisuan(string):
    y = int(string.split('-')[0])
    m = int(string.split('-')[1])
    d = int(string.split('-')[2])
    if d > 25 and m < 12:
        m = m + 1
    elif d > 25 and m == 12 :
        m = 1
        y = y + 1
    else :
        pass
    qijian = str(y)+'-'+str(m).zfill(2)
    return qijian
lpj = lpj.assign(期间=lpj['入库日期'].map(lambda x: qijianJisuan(x)))


In [6]:
lpj['标准日期'] = lpj['入库日期']
lpj['标准日期'] = lpj['标准日期'].astype('datetime64[ns]')
lpj['入库日期'] = lpj['入库日期'].astype('datetime64[ns]')

In [7]:
in_subject = ['入库单号', '入库日期', '相关单位', '单据附注', '货品编码', '货品名称', '规格', '所属类别', '单位', '入库数量', '单价', '金额', '备注',
                       '期间', '标准日期']

index_col = ['标准日期', '入库单号', '相关单位']

In [8]:
olddata = pd.read_excel(lpj_fname, sheetname,dtype = {'单据号':str})
data = pd.concat([olddata, lpj])
col_names = data.columns.to_list()
data.drop_duplicates(subset=in_subject, keep='first', inplace=True)
data = data.set_index(index_col)                   #按日期索引
data = data.sort_values(by=index_col)
data = data.reset_index()                               #取消索引
data = data[col_names]                               #按原来列顺序
data

,入库单号,入库日期,相关单位,单据附注,货品编码,货品名称,规格,所属类别,单位,入库数量,单价,金额,备注,期间,标准日期
0,RK1812260089,2018-12-26,欣华誉,NaN,YYLB0020,环乙酮,NaN,YYLB医用劳保品,公斤,180.0,11.8,2124.0,NaN,2019-01,2018-12-26
1,RK1812260090,2018-12-26,长昊,NaN,RBLD光轴30L1200,光轴,30L1200,RBLD柔版联动线配件,支,4.0,72.0,288.0,NaN,2019-01,2018-12-26
2,RK1812260090,2018-12-26,长昊,NaN,ZC轴承国产LM30UU,轴承国产LM30UU,LM30UU,ZC轴承国产,套,4.0,20.0,80.0,NaN,2019-01,2018-12-26
3,RK1812260091,2018-12-26,上海天岑机械,NaN,SYSG上光机涂布辊,上光机涂布辊,215x1075,SYSG丝印上光机配件,件,1.0,6700.0,6700.0,NaN,2019-01,2018-12-26
4,RK1812260092,2018-12-26,沅键五金商行,NaN,GJL胶枪,胶枪,NaN,GJL工具类,把,1.0,60.0,60.0,NaN,2019-01,2018-12-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4780,RK2306140031,2023-06-14,鑫盛物资商行,NaN,铁丝,铁丝,18号,ZMDQ照明电器及设备,卷,1.0,48.0,48.0,NaN,2023-06,2023-06-14
4781,RK2306140032,2023-06-14,恒发,NaN,YZ00010,0﹟柴油,NaN,YZL油脂类,升,1800.0,6.8,12240.0,NaN,2023-06,2023-06-14
4782,RK2306150033,2023-06-15,淘宝网,NaN,JYPJ压纸轮,压纸轮,NaN,JYPJ胶印配件,件,6.0,45.0,270.0,NaN,2023-06,2023-06-15
4783,RK2306160034,2023-06-16,丰页,NaN,儿童订弯脚组1,儿童订弯脚组,52/8,RBLD柔版联动线配件,个,8.0,190.0,1520.0,NaN,2023-06,2023-06-16


In [25]:
data['入库日期'] = data['入库日期'].astype('str')
data['标准日期'] = data['标准日期'].astype('str')
data.dtypes

入库单号     object
入库日期     object
相关单位     object
单据附注    float64
货品编码     object
货品名称     object
规格       object
所属类别     object
单位       object
入库数量    float64
单价      float64
金额      float64
备注       object
期间       object
标准日期     object
dtype: object

In [26]:
data

,入库单号,入库日期,相关单位,单据附注,货品编码,货品名称,规格,所属类别,单位,入库数量,单价,金额,备注,期间,标准日期
0,RK1812260089,2018-12-26,欣华誉,NaN,YYLB0020,环乙酮,NaN,YYLB医用劳保品,公斤,180.0,11.8,2124.0,NaN,2019-01,2018-12-26
1,RK1812260090,2018-12-26,长昊,NaN,RBLD光轴30L1200,光轴,30L1200,RBLD柔版联动线配件,支,4.0,72.0,288.0,NaN,2019-01,2018-12-26
2,RK1812260090,2018-12-26,长昊,NaN,ZC轴承国产LM30UU,轴承国产LM30UU,LM30UU,ZC轴承国产,套,4.0,20.0,80.0,NaN,2019-01,2018-12-26
3,RK1812260091,2018-12-26,上海天岑机械,NaN,SYSG上光机涂布辊,上光机涂布辊,215x1075,SYSG丝印上光机配件,件,1.0,6700.0,6700.0,NaN,2019-01,2018-12-26
4,RK1812260092,2018-12-26,沅键五金商行,NaN,GJL胶枪,胶枪,NaN,GJL工具类,把,1.0,60.0,60.0,NaN,2019-01,2018-12-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4780,RK2306140031,2023-06-14,鑫盛物资商行,NaN,铁丝,铁丝,18号,ZMDQ照明电器及设备,卷,1.0,48.0,48.0,NaN,2023-06,2023-06-14
4781,RK2306140032,2023-06-14,恒发,NaN,YZ00010,0﹟柴油,NaN,YZL油脂类,升,1800.0,6.8,12240.0,NaN,2023-06,2023-06-14
4782,RK2306150033,2023-06-15,淘宝网,NaN,JYPJ压纸轮,压纸轮,NaN,JYPJ胶印配件,件,6.0,45.0,270.0,NaN,2023-06,2023-06-15
4783,RK2306160034,2023-06-16,丰页,NaN,儿童订弯脚组1,儿童订弯脚组,52/8,RBLD柔版联动线配件,个,8.0,190.0,1520.0,NaN,2023-06,2023-06-16


In [27]:
app = xw.App(visible=False, add_book=False)
wb = app.books.open(lpj_fname)
ws = wb.sheets[sheetname]
ws.clear_contents()
wb.save()
wb.close()
app.quit()


In [28]:
with pd.ExcelWriter(lpj_fname, engine='openpyxl',datetime_format='yyyy-m-d', mode='a', if_sheet_exists='overlay') as writer:
        data.to_excel(writer, sheetname, index=False)



In [ ]:
desktopPath = r'D:\ribaobiao'
os.chdir(desktopPath)
filenames = os.listdir(desktopPath)
lpj_files = [os.path.join(desktopPath,i) for i in filenames if fnmatch(i, '*统计*.xls*')]
lpj_files

In [ ]:
msg = '请选择日期输入的方式：'
choice = buttonbox(msg, choices=['自动', '手动','singleZhi','singleBaiyun'])
if choice == '自动':
    today = date.today()
    start_riqi, end_riqi = startriqiEndriqi(today)


In [ ]:
datas = []
for filename in lpj_files:
    xlsxname = xlsToXlsx(filename)
    data = pd.read_excel(xlsxname)

    if data.columns[0]=='退货单号':
        data.rename(columns={'退货单号':'入库单号','退货日期': '入库日期','退货数量':'入库数量'},inplace=True)
        data['入库数量'] = data['入库数量'] * -1
        data['金额'] = data['金额'] * -1
    else :
        pass
    data.dropna(subset=['入库单号'], inplace=True)  # 删除入库单号列中的有空值的行

    datas.append(data)
lpj = pd.concat(datas)
lpj






In [ ]:
for j in ['期间','标准日期']:
    lpj[j] = lpj['入库日期']
lpj['标准日期'] = lpj['标准日期'].astype('datetime64[ns]')
lpj = lpj.assign(期间=lpj['入库日期'].map(lambda x: qijianJisuan(x)))
lpj['入库日期'] = lpj['入库日期'].astype('datetime64[ns]')
lpj

In [ ]:
app = xw.App(visible=False, add_book=False)
wb = app.books.open(lpj_fname)
ws = wb.sheets[sheetname]
olddata = pd.read_excel(lpj_fname, sheetname,dtype = {'单据号':str})
data = pd.concat([olddata, newdata])
col_names = data.columns.to_list()
data.drop_duplicates(subset=in_subject, keep='first', inplace=True)
data = data.set_index(index_col)                   #按日期索引
data = data.sort_values(by=index_col)
data = data.reset_index()                               #取消索引
data = data[col_names]
data

In [ ]:
def concatAndQuchong(fname,sheetname,newdata,in_subject,index_col=['日期','单据号', '供货单位'] ):
    app = xw.App(visible=False, add_book=False)
    wb = app.books.open(fname)
    ws = wb.sheets[sheetname]
    olddata = pd.read_excel(fname, sheetname,dtype = {'单据号':str})
    data = pd.concat([olddata, newdata])
    col_names = data.columns.to_list()
    data.drop_duplicates(subset=in_subject, keep='first', inplace=True)
    data = data.set_index(index_col)                   #按日期索引
    data = data.sort_values(by=index_col)
    data = data.reset_index()                               #取消索引
    data = data[col_names]                               #按原来列顺序
    # first_col = data.columns[0]                   #取第一列
    # data = data.set_index(first_col)      #按首列索引
    ws.clear_contents()
    # df_temp = data.applymap(lambda x: str(x) if isinstance(x, datetime.time) else x)
    # ws.range('A1').options(pd.DataFrame, index=False).value = df_temp
    data1 = data.astype(data.dtypes.replace({'datetime64[ns]': str}))
    ws.range('A1').options(pd.DataFrame, index=False).value = data1
    wb.save()
    wb.close()
    app.quit()
    return fname

In [ ]:
lpj_fname = r'F:\a00nutstore\006\zw\lingpeijian\零配件实时入库2020.xlsx'
sheetname = 'ssrk'

In [ ]:
app = xw.App(visible=False, add_book=False)
wb = app.books.open(fname)
ws = wb.sheets[sheetname]
olddata = pd.DataFrame(pd.read_excel(fname, sheetname,dtype = {'单据号':str}))
data = pd.concat([olddata, newdata])

In [ ]:
F:\a00nutstore\006\zw\lingpeijian\零配件实时入库2020.xlsx

In [ ]:
lpj_fname = r'F:\a00nutstore\006\zw\lingpeijian\零配件实时入库2020.xlsx'

In [ ]:
in_subject = ['入库单号', '入库日期', '相关单位', '单据附注', '货品编码', '货品名称', '规格', '所属类别', '单位', '入库数量', '单价', '金额', '备注',
                           '期间', '标准日期']
index_col = ['入库日期', '入库单号', '相关单位']
app = xw.App(visible=False, add_book=False)
wb = app.books.open(lpj_fname)
ws = wb.sheets['ssrk']
olddata = pd.DataFrame(pd.read_excel(lpj_fname,'ssrk',dtype = {'单据号':str}))
data = pd.concat([olddata,lpj])
col_names = data.columns.to_list()
data.drop_duplicates(subset=in_subject, keep='first', inplace=True)
data = data.set_index(index_col)                   #按日期索引
data = data.sort_values(by=index_col)
data = data.reset_index()                               #取消索引
data = data[col_names]                               #按原来列顺序
# first_col = data.columns[0]                   #取第一列
# data = data.set_index(first_col)      #按首列索引
ws.clear_contents()
ws.range('A1').options(pd.DataFrame, index=False).value = data
wb.save()
wb.close()
app.quit()

In [ ]:
olddata = pd.DataFrame(pd.read_excel(lpj_fname,'ssrk',dtype = {'单据号':str}))
olddata.dtypes

In [ ]:
olddata

In [ ]:
a = pd.concat([olddata,lpj])
a.dtypes

In [ ]:
data

In [ ]:
data.dtypes

In [ ]:
data.to_excel('111.xlsx',index = False)

In [ ]:
import datetime

In [ ]:
data['入库日期'] = data['入库日期'].apply(lambda x: str(x) if isinstance(x, datetime.time) else x)
data

In [ ]:
data.dtypes

In [ ]:
df_temp = data.applymap(lambda x: str(x) if isinstance(x, datetime.time) else x)
df_temp
# ws.range('A1').options(pd.DataFrame, index=False).value = df_temp

In [ ]:
data.astype({'入库日期':"str"})

In [ ]:
data.dtypes

In [ ]:
data1 = data.astype(data.dtypes.replace({'datetime64[ns]': str}))

In [ ]:
data1.dtypes

In [31]:
lpj_fname = r'F:\a00nutstore\006\zw\lingpeijian\零配件实时入库2020.xlsx'
sheetname = 'ssrk'
data = pd.read_excel(lpj_fname,sheet_name=sheetname)
data.dtypes

入库单号     object
入库日期     object
相关单位     object
单据附注    float64
货品编码     object
货品名称     object
规格       object
所属类别     object
单位       object
入库数量    float64
单价      float64
金额      float64
备注       object
期间       object
标准日期     object
dtype: object

In [37]:
data['标准日期'] = data['标准日期'].astype('datetime64[ns]')
data.dtypes

入库单号            object
入库日期            object
相关单位            object
单据附注           float64
货品编码            object
货品名称            object
规格              object
所属类别            object
单位              object
入库数量           float64
单价             float64
金额             float64
备注              object
期间              object
标准日期    datetime64[ns]
dtype: object

In [36]:
df_temp = data.applymap(lambda x: str(x) if pd.api.types.is_datetime64_dtype else x)
df_temp.dtypes

入库单号    object
入库日期    object
相关单位    object
单据附注    object
货品编码    object
货品名称    object
规格      object
所属类别    object
单位      object
入库数量    object
单价      object
金额      object
备注      object
期间      object
标准日期    object
dtype: object

In [38]:
data1 = data.astype(data.dtypes.replace({'datetime64[ns]': str}))
data1.dtypes

入库单号     object
入库日期     object
相关单位     object
单据附注    float64
货品编码     object
货品名称     object
规格       object
所属类别     object
单位       object
入库数量    float64
单价      float64
金额      float64
备注       object
期间       object
标准日期     object
dtype: object

In [42]:
filename = r'F:\repos\fish\原材料实时流水账_20230616085039.xlsx'
df = pd.read_excel(filename, '流水账')
df.dtypes

日期           datetime64[ns]
单据号                   int64
供货单位                 object
品名                   object
单位                   object
入库数量                float64
入库单价                float64
入库金额                float64
cwName               object
priceName            object
期间                   object
送货日期         datetime64[ns]
白云期间                 object
令数                  float64
吨数                  float64
令价                  float64
吨价                  float64
吨价0                 float64
记账                   object
单价                  float64
dtype: object

In [47]:
filename1 = r'F:\repos\fish\原材料实时流水账.xlsx'
df1 = pd.read_excel(filename1, '流水账')
df1.dtypes

日期           datetime64[ns]
单据号                   int64
供货单位                 object
品名                   object
单位                   object
入库数量                float64
入库单价                float64
入库金额                float64
cwName               object
priceName            object
期间                   object
送货日期         datetime64[ns]
白云期间                 object
令数                  float64
吨数                  float64
令价                  float64
吨价                  float64
吨价0                 float64
记账                   object
单价                  float64
dtype: object

In [46]:
filename1 = r'F:\repos\fish\零配件实时入库2020.xlsx'
df3 = pd.read_excel(filename1, 'ssrk')
df3.dtypes

入库单号            object
入库日期    datetime64[ns]
相关单位            object
单据附注           float64
货品编码            object
货品名称            object
规格              object
所属类别            object
单位              object
入库数量           float64
单价             float64
金额             float64
备注              object
期间              object
标准日期    datetime64[ns]
dtype: object

In [ ]:
from datetime import date, datetime  
df = pd.DataFrame(
    [
        [date(2014, 1, 31), date(1999, 9, 24)],
        [datetime(1998, 5, 26, 23, 33, 4), datetime(2014, 2, 28, 13, 5, 13)],
    ],
    index=["Date", "Datetime"],
    columns=["X", "Y"],
)  
with pd.ExcelWriter(
    "path_to_file.xlsx",
    date_format="YYYY-MM-DD",
    datetime_format="YYYY-MM-DD HH:MM:SS"
) as writer:
    df.to_excel(writer)  

In [13]:
from datetime import date, datetime  
df1 = pd.DataFrame(
    [
        [date(2014, 1, 31), date(1999, 9, 24)],
        [datetime(1998, 5, 26, 23, 33, 4), datetime(2014, 2, 28, 13, 5, 13)],
    ],
    index=["Date", "Datetime"],
    columns=["X", "Y"],
)  
df1

,X,Y
Date,2014-01-31 00:00:00,1999-09-24 00:00:00
Datetime,1998-05-26 23:33:04,2014-02-28 13:05:13


In [17]:
with pd.ExcelWriter(
    "path_to_file.xlsx",engine = 'openpyxl',
    date_format="YYYY-MM-DD",
    datetime_format="YYYY-MM-DD HH:MM:SS",mode='a',
                    if_sheet_exists='replace'
) as writer:
    df1.to_excel(writer)  